In [0]:
%%sh
PGPASSWORD='duI3wInpB@g82os2' psql -h 10.2.10.132 -p 5432 -Ucitus1_datapipeline -d aa_citus_db << EOF
SET search_path = 'store';
-- \d
--select * from store.store_market_size_fact_v1 where date='2020-10-01' and device_code= 'ios-tablet' and country_code='CA' and category_id = 100009 order by est_market_size_revenue
select * from store.store_market_size_fact_v1 where date between '2020-01-03' and '2020-01-09' and country_code='US' and category_id in (100000, 400000) limit 10;
EOF

In [0]:


def platform_feed_to_metric(platform, feed):
    mapping = [
        ['ios', 0, 'iphone_free'],
        ['ios',1, 'iphone_paid'],
        ['ios',2, 'iphone_revenue'],
        ['ios',101,'ipad_free'],
        ['ios',100,'ipad_paid'],
        ['ios',102,'ipad_revenue'],
        ['android',0,'est_free_app_download'],
        ['android',1,'est_paid_app_download'],
        ['android',2,'est_revenue'],
    ]
    return [x for x in mapping if (x[0], x[1]) == (platform, feed)][0][2]
    
platform_feed_to_metric("ios",0)

In [0]:


import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F


start_week = "2015-01-01"
end_week = "2015-02-01"

real_date1 = datetime.date(*[int(x) for x in start_week.split('-')])
real_date2 = datetime.date(*[int(x) for x in end_week.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(str(real_date1 + datetime.timedelta(days)))

dates_temp=list()
sar_list=list()
for days in xrange(date_range.days):
    dates_temp.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates_temp:
            temp.append(dates_temp.pop())
        sar_list.append(str(real_date1 + datetime.timedelta(days)))


daily_csv_schema = T.StructType(
    [
        T.StructField("store_id", T.IntegerType(), True),
        T.StructField("date", T.StringType(), True),
        T.StructField("platform_id", T.IntegerType(), True),
        T.StructField("vertical", T.IntegerType(), True),
        T.StructField("feed", T.IntegerType(), True),
        T.StructField("id", T.LongType(), True),
        T.StructField("est", T.IntegerType(), True),
        T.StructField("category", T.IntegerType(), True),
        T.StructField("rank", T.IntegerType(), True)
    ]
)



weekly_csv_schema = T.StructType(
    [
        T.StructField("id", T.LongType(), True),
        T.StructField("store_id", T.IntegerType(), True),
        T.StructField("category", T.IntegerType(), True),
        T.StructField("start_date", T.StringType(), True),
        T.StructField("end_date", T.StringType(), True),
        T.StructField("iphone_free", T.IntegerType(), True),
        T.StructField("iphone_paid", T.IntegerType(), True),
        T.StructField("iphone_revenue", T.IntegerType(), True),
        T.StructField("ipad_free", T.LongType(), True),
        T.StructField("ipad_paid", T.IntegerType(), True),
        T.StructField("ipad_revenue", T.IntegerType(), True)
    ]
)
 
monthly_csv_schema = T.StructType(
    [
        T.StructField("id", T.LongType(), True),
        T.StructField("store_id", T.IntegerType(), True),
        T.StructField("category", T.IntegerType(), True),
        T.StructField("year", T.StringType(), True),
        T.StructField("month", T.StringType(), True),
        T.StructField("iphone_free", T.IntegerType(), True),
        T.StructField("iphone_paid", T.IntegerType(), True),
        T.StructField("iphone_revenue", T.IntegerType(), True),
        T.StructField("ipad_free", T.LongType(), True),
        T.StructField("ipad_paid", T.IntegerType(), True),
        T.StructField("ipad_revenue", T.IntegerType(), True)
    ]
)
       

print 'test date'
print dates
print sar_list
 

df_1 = spark.read.option("basePath", "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/").schema(daily_csv_schema).csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{%s}/ios/sbe_est_app/*/" % ",".join(dates), sep="\t").cache()
df_2 = spark.read.option("basePath", "s3://b2c-prod-dca-store-estimates/store_est/v_final/MONTH/").schema(monthly_csv_schema).csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/MONTH/{%s}/ios/sbe_est_app/*/" % dates[-1], sep="\t").cache()
df_3 = spark.read.option("basePath", "s3://b2c-prod-dca-store-estimates/store_est/v_final/WEEK/").schema(weekly_csv_schema).csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/WEEK/{%s}/ios/sbe_est_app/*/" % ",".join(sar_list), sep="\t").cache()

df_1.createOrReplaceTempView("daily_data")
df_2.createOrReplaceTempView("monthly_data")
df_3.createOrReplaceTempView("weekly_data")



In [0]:

spark.sql("select * from daily_data where id=378736412 and store_id=143462 and category=100 ").show(2)
spark.sql("select * from weekly_data where id=378736412 and store_id=143462 and category=100 ").show()
spark.sql("select * from monthly_data where id=378736412 and store_id=143462 and category=100 ").show()

In [0]:

spark.sql("select * from daily_data where id=585544408 and store_id=143496 and category=7019 and feed=101 ").show(2) # ipad free
spark.sql("select * from weekly_data where id=585544408 and store_id=143496 and category=7019").show()
spark.sql("select * from monthly_data where id=585544408 and store_id=143496 and category=7019 ").show()

In [0]:

test_df = spark.sql('''
SELECT * 
FROM   ( 
                SELECT   id, 
                         store_id, 
                         category, 
                         platform_id, 
                         vertical, 
                         feed, 
                         Sum(est) AS est
                FROM     daily_data 
                WHERE    feed IN ( 0, 
                                  1, 
                                  2, 
                                  101, 
                                  100, 
                                  102 ) 
                GROUP BY id, 
                         store_id, 
                         category, 
                         platform_id, 
                         vertical, 
                         feed
                          ) PIVOT ( max(est) FOR feed IN (0, 
                                                          1, 
                                                          2, 
                                                          101, 
                                                          100, 
                                                          102) ) ''').withColumnRenamed("0", platform_feed_to_metric('ios',0)).withColumnRenamed("1", platform_feed_to_metric('ios',1)).withColumnRenamed("2", platform_feed_to_metric('ios',2)).withColumnRenamed("101", platform_feed_to_metric('ios',101)).withColumnRenamed("100", platform_feed_to_metric('ios',100)).withColumnRenamed("102", platform_feed_to_metric('ios',102)).na.fill(0).cache()
test_df.createOrReplaceTempView("transfer")



In [0]:

spark.sql("select * from transfer").show(2)
diff_df_1 = spark.sql("select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from transfer except all select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from monthly_data ").cache()
diff_df_2 = spark.sql("select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from monthly_data except all select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from transfer").cache()
print diff_df_1.take(10)
print diff_df_2.take(10)


In [0]:

spark.sql("select id, category, sum(est) from daily_data where id=585544408 and store_id=143496 and feed=0 group by category, id ").show(32) # ipad free
spark.sql("select id, category, est, date from daily_data where id=585544408 and store_id=143496 and feed=0  ").show(500) # ipad free

spark.sql("select * from weekly_data where id=585544408 and store_id=143496 ").show()
spark.sql("select * from monthly_data where id=585544408 and store_id=143496  ").show()










In [0]:

new_transfor_df = spark.sql('''
SELECT
  *
FROM
  (
    SELECT
      id,
      Sum(est) AS est,
      category,
      store_id,
      platform_id,
      feed,
      vertical
    FROM
      (
        SELECT
          DISTINCT d1.id,
          d1.est,
          d1.store_id,
          d1.date,
          d1.feed,
          d1.vertical,
          d1.platform_id,
          d2.category
        FROM
          daily_data AS d1
          JOIN daily_data AS d2 
          ON d1.id = d2.id
          AND d1.store_id = d2.store_id
          AND d1.feed = d2.feed
          AND d1.vertical = d2.vertical
          AND d1.platform_id = d2.platform_id
      ) AS t
    WHERE
      feed IN (
        0,
        1,
        2,
        101,
        100,
        102
      )
    GROUP BY
      id,
      store_id,
      category,
      platform_id,
      vertical,
      feed
  ) PIVOT (
    max(est) FOR feed IN (
      0,
      1,
      2,
      101,
      100,
      102
    )
  )
''').withColumnRenamed("0", platform_feed_to_metric('ios',0)).withColumnRenamed("1", platform_feed_to_metric('ios',1)).withColumnRenamed("2", platform_feed_to_metric('ios',2)).withColumnRenamed("101", platform_feed_to_metric('ios',101)).withColumnRenamed("100", platform_feed_to_metric('ios',100)).withColumnRenamed("102", platform_feed_to_metric('ios',102)).na.fill(0).cache()
                                                        
new_transfor_df.createOrReplaceTempView("transfer_new")

spark.sql("select * from transfer_new where id=585544408 and store_id=143496").show()


In [0]:

test_df = spark.sql('''
SELECT * 
FROM   ( 
                SELECT   id, 
                         store_id, 
                         category, 
                         platform_id, 
                         vertical, 
                         feed, 
                         Sum(est) AS est
                FROM     
                        (select 
                        distinct 
                        id, 
                         store_id, 
                         platform_id, 
                         vertical, 
                         feed,
                         est from daily_data) 
                         as distinct_data
                WHERE    feed IN ( 0, 
                                  1, 
                                  2, 
                                  101, 
                                  100, 
                                  102 ) 
                AND  id=585544408 and store_id=143496 and feed=0 
                GROUP BY id, 
                         store_id, 
                         category, 
                         platform_id, 
                         vertical, 
                         feed
                          ) PIVOT ( max(est) FOR feed IN (0, 
                                                          1, 
                                                          2, 
                                                          101, 
                                                          100, 
                                                          102) ) ''').withColumnRenamed("0", platform_feed_to_metric('ios',0)).withColumnRenamed("1", platform_feed_to_metric('ios',1)).withColumnRenamed("2", platform_feed_to_metric('ios',2)).withColumnRenamed("101", platform_feed_to_metric('ios',101)).withColumnRenamed("100", platform_feed_to_metric('ios',100)).withColumnRenamed("102", platform_feed_to_metric('ios',102)).na.fill(0).cache()
test_df.createOrReplaceTempView("transfer")



In [0]:

print spark.sql("select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from transfer_new where category =100 except select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from monthly_data where category =100 order by iphone_paid desc ").show(200)

print spark.sql("select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from monthly_data where category =100 except select id, category, store_id, iphone_free, iphone_paid, iphone_revenue, ipad_free, ipad_paid, ipad_revenue from transfer_new where category =100  order by iphone_paid desc ").show(200)





In [0]:

# spark.sql("select * from daily_data where id=891453663 and store_id=143441 and date between '2015-01-03' and '2015-01-10' and feed = 1 and category in (100, 36) order by category desc ").show()
# spark.sql("select * from weekly_data where id=891453663 and store_id=143441 and category=100   ").show(2)
spark.sql("select * from monthly_data where id=891453663 and store_id=143441  ").show()


In [0]:

+--------+----------+-----------+--------+----+---------+---+--------+----+
|store_id|      date|platform_id|vertical|feed|       id|est|category|rank|
+--------+----------+-----------+--------+----+---------+---+--------+----+
|  143441|2015-01-09|          1|       0|   1|891453663|1|     100|  43|
|  143441|2015-01-03|          1|       0|   1|891453663|1|     100|  38|
|  143441|2015-01-08|          1|       0|   1|891453663|1|     100|  31|
|  143441|2015-01-10|          1|       0|   1|891453663|1|     100|  43|
|  143441|2015-01-05|          1|       0|   1|891453663|1|     100|  75|
|  143441|2015-01-07|          1|       0|   1|891453663|1|     100| 103|
|  143441|2015-01-04|          1|       0|   1|891453663|1|     100|  49|
|  143441|2015-01-06|          1|       0|   1|891453663|1|     100|  79|
|  143441|2015-01-03|          1|       0|   1|891453663|1|      36| 106|
|  143441|2015-01-08|          1|       0|   1|891453663|1|      36|  83|
+--------+----------+-----------+--------+----+---------+---+--------+----+

weekly = 7 36
weekly = 7 100

weekly = 2 36
weekly = 7 100


In [0]:

print spark.sql("select * from transfer_new where id=891453663 and store_id=143441").show()

In [0]:

14237344.000/39529475.000